<a href="https://colab.research.google.com/github/danmartin25/Hockey_Model/blob/main/Hockey_Model_Mark_I.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Notes**
1. On-ice EV xG+/-

2. On-ice EV G+/-

3. On-ice PP G+/- above average

4. On-ice SH G+/- above average

5. GSAx

6. Individual points above average (depending on position and role)

In [1]:
# Import Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#Import Even-Strength On-Ice Totals Data
skater_EV_totals_raw = pd.read_csv('https://raw.githubusercontent.com/danmartin25/Hockey_Model/main/EV%20On-Ice%20Totals.csv')
print(skater_EV_totals_raw)
#skater_EV_totals_raw.head()

                 Player Season Team Position  GP      TOI    GF%    SF%  \
0            A.J. Greer  21-22  N.J        L   9    70.45  50.50  57.90   
1          Aaron Ekblad  19-20  FLA        D  67  1220.55  56.75  54.15   
2          Aaron Ekblad  20-21  FLA        D  35   603.07  55.98  55.00   
3          Aaron Ekblad  21-22  FLA        D  61  1047.80  63.35  58.86   
4            Aaron Ness  19-20  ARI        D  24   289.15  49.44  51.48   
...                 ...    ...  ...      ...  ..      ...    ...    ...   
2650        Zdeno Chara  19-20  BOS        D  68  1166.42  62.20  49.01   
2651        Zdeno Chara  20-21  WSH        D  55   813.33  51.49  49.44   
2652        Zdeno Chara  21-22  NYI        D  72  1191.52  53.26  44.96   
2653  Zemgus Girgensons  19-20  BUF        C  69   783.73  45.67  47.49   
2654  Zemgus Girgensons  21-22  BUF      C/L  56   668.30  44.00  47.21   

        FF%    CF%  ...       CA    xGF    xGA     G±      S±      F±      C±  \
0     57.33  56.89

In [3]:
#Restrict Totals Data to GP,TOI,GF%,xGF%,GF,GA,xGF,xGA
skater_EV_totals = skater_EV_totals_raw.loc[:,['Player','Season','Team','Position','GP','TOI','GF%','xGF%','GF','GA','xGF','xGA']]
#skater_EV_totals

In [4]:
#Add columns for G+/-,xG+/-
skater_EV_totals['G+/-'] = skater_EV_totals['GF'] - skater_EV_totals['GA']
skater_EV_totals['xG+/-'] = skater_EV_totals['xGF'] - skater_EV_totals['xGA']
#skater_EV_totals

In [5]:
#Import PP On-Ice Rates Data
skater_PP_totals_raw = pd.read_csv('https://raw.githubusercontent.com/danmartin25/Hockey_Model/main/PP%20On-Ice%20Totals.csv')
print(skater_PP_totals_raw)
#skater_PP_totals_raw.head()

                 Player Season Team Position  GP     TOI     GF%    SF%  \
0          Aaron Ekblad  19-20  FLA        D  67   95.90   92.22  81.87   
1          Aaron Ekblad  20-21  FLA        D  35  132.75   91.82  87.33   
2          Aaron Ekblad  21-22  FLA        D  61  214.78   92.51  86.16   
3          Adam Boqvist  19-20  CHI        D  41   87.02   88.20  81.06   
4          Adam Boqvist  20-21  CHI        D  35  118.65   82.03  75.97   
...                 ...    ...  ...      ...  ..     ...     ...    ...   
1303      Zach Werenski  19-20  CBJ        D  63  171.17   80.98  84.22   
1304      Zach Werenski  20-21  CBJ        D  33   73.60  100.00  81.71   
1305      Zach Werenski  21-22  CBJ        D  64  215.48   87.27  79.95   
1306       Zack Kassian  19-20  EDM        R  58   23.18   54.38  91.63   
1307  Zemgus Girgensons  21-22  BUF      C/L  55   54.37  100.00  83.74   

        FF%    CF%  ...     CA    xGF   xGA     G±      S±      F±      C±  \
0     84.52  85.63  .

In [6]:
#Reduce PP On-Ice Totals Data
skater_PP_totals = skater_PP_totals_raw.loc[:,['Player','Season','Team','Position','GP','TOI','GF%','xGF%','GF','GA','xGF','xGA']]
#skater_PP_totals

In [7]:
#Add columns for G+/-,xG+/-,G/s,xG/s
skater_PP_totals['G+/-'] = skater_PP_totals['GF'] - skater_PP_totals['GA']
skater_PP_totals['xG+/-'] = skater_PP_totals['xGF'] - skater_PP_totals['xGA']
#skater_PP_totals

In [8]:
#Add column for mean G+/- and xG+/-
G_mean = skater_PP_totals['G+/-'].mean()
xG_mean = skater_PP_totals['xG+/-'].mean()
#G_mean
#xG_mean

In [9]:
#Add column for stats above average for G+/- and xG+/-
skater_PP_totals['GAA'] = skater_PP_totals['G+/-'] - G_mean
skater_PP_totals['xGAA'] = skater_PP_totals['xG+/-'] - xG_mean
#skater_PP_totals

In [10]:
#Drop columns
skater_PP_totals = skater_PP_totals.drop(columns = ['xGF%','xGF','xGA','xG+/-','xGAA'])
#skater_PP_totals

In [11]:
#Import SH On-Ice Totals Data
skater_SH_totals_raw = pd.read_csv('https://raw.githubusercontent.com/danmartin25/Hockey_Model/main/SH%20On-Ice%20Totals.csv')
#print(skater_SH_totals_raw)
#skater_SH_totals_raw.head()

In [12]:
#Reduce SH On-Ice Totals Data
skater_SH_totals = skater_SH_totals_raw.loc[:,['Player','Season','Team','Position','GP','TOI','GF%','xGF%','GF','GA','xGF','xGA']]
#skater_SH_totals

In [13]:
#Add columns for G+/-,xG+/-
skater_SH_totals['G+/-'] = skater_SH_totals['GF'] - skater_SH_totals['GA']
skater_SH_totals['xG+/-'] = skater_SH_totals['xGF'] - skater_SH_totals['xGA']
#skater_SH_totals

In [14]:
#Add column for mean G+/- and xG+/-
G_mean = skater_SH_totals['G+/-'].mean()
xG_mean = skater_SH_totals['xG+/-'].mean()
#G_mean
#xG_mean

In [15]:
#Add column for stats above average for G+/- and xG+/-
skater_SH_totals['GAA'] = skater_SH_totals['G+/-'] - G_mean
skater_SH_totals['xGAA'] = skater_SH_totals['xG+/-'] - xG_mean
#skater_SH_totals

In [16]:
skater_SH_totals = skater_SH_totals.drop(columns = ['xGF%','xGF','xGA','xG+/-','xGAA'])
#skater_SH_totals

In [17]:
#Import Player Box Stats
skater_box_totals_raw = pd.read_csv('https://raw.githubusercontent.com/danmartin25/Hockey_Model/main/Player%20Stats%20Box.csv')
#print(skater_box_totals_raw)
#skater_box_totals_raw.head()

In [18]:
#Reduce Data Stats
skater_box_totals = skater_box_totals_raw.loc[:,['Player','Season','Team','Position','GP','TOI','G','A1','A2','Points']]
skater_box_totals.head()

,Player,Season,Team,Position,GP,TOI,G,A1,A2,Points
0,A.J. Greer,21-22,N.J,L,9,70.70,1,0,1,2
1,Aaron Ekblad,19-20,FLA,D,67,1537.92,5,23,13,41
2,Aaron Ekblad,20-21,FLA,D,35,878.18,11,5,6,22
3,Aaron Ekblad,21-22,FLA,D,61,1519.70,15,13,29,57
4,Aaron Ness,19-20,ARI,D,24,300.83,0,1,0,1


In [19]:
#Look at positions to get correct F/D average points
#print(skater_box_totals['Position'].unique())
#print(skater_box_totals.count())

forwards_table = skater_box_totals.loc[(skater_box_totals.Position == "L") | (skater_box_totals.Position == "C") | (skater_box_totals.Position == "C/L") | (skater_box_totals.Position == "R") | (skater_box_totals.Position == "L/R") | (skater_box_totals.Position == "C/R")]
#forwards_table.head(10)
#print(forwards_table.count())

defensemen_table = skater_box_totals.loc[(skater_box_totals.Position == "D") | (skater_box_totals.Position == "D/L")]
#defensemen_table.head(10)
#print(defensemen_table.count())

In [20]:
#Get Mean of Points
Points_mean = skater_box_totals['Points'].mean()
#print(Points_mean)

Forwards_mean = forwards_table['Points'].mean()
#print(Forwards_mean)

Defensemen_mean = defensemen_table['Points'].mean()
#print(Defensemen_mean)

In [21]:
#Get PAA
#skater_box_totals['PAA'] = skater_box_totals['Points'] - Points_mean
#skater_box_totals

#Get PAA for Fwds and Dmen and combine

forwards_table['PAA'] = forwards_table['Points'] - Forwards_mean
forwards_table.head(10)

defensemen_table['PAA'] = defensemen_table['Points'] - Defensemen_mean
defensemen_table.head(10)


#Add dataframes back together and sort again

concat_frames = [forwards_table, defensemen_table]
skater_box_totals = pd.concat(concat_frames)
skater_box_totals = skater_box_totals.sort_index()
#skater_box_totals.head(10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [22]:
combined_player_df = pd.merge(skater_EV_totals, skater_PP_totals, how="left", on=["Player", "Player"])
#combined_player_df

In [23]:
combined_player_df = combined_player_df.drop(columns = ['GF%_x','GF_x','GA_x','Season_y','Team_y','Position_y','GP_y','GF%_y','GF_y','GA_y','G+/-_y'])
#combined_player_df

In [24]:
#Merge EV,PP,SH tables together
full_player_totals = pd.merge(combined_player_df, skater_SH_totals, how="left", on=["Player", "Player"])
#full_player_totals.head(10)

In [25]:
#Clean up data
full_player_totals = full_player_totals.drop(columns = ['Season','Team','Position','GP','GF%','GF','GA','G+/-'])
#full_player_totals.head(10)

In [26]:
#Merge EV,PP,SH and Points tables together
full_player_totals = pd.merge(full_player_totals, skater_box_totals, how="left", on=["Player", "Player"])
#full_player_totals.head(10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Passing 'suffixes' which cause duplicate columns {'TOI_x'} in the result is deprecated and will raise a MergeError in a future version.
  


In [27]:
#Drop Repeated Columns
full_player_totals = full_player_totals.drop(columns = ['Season','Team','Position','GP','G','A1','A2'])
#full_player_totals.head(10)


In [28]:
#Rename Columns
full_player_totals = full_player_totals.rename(columns = {'Season_x':'Season','Team_x':'Team','Position_x':'Position','GP_x':'GP','TOI_x':'TOI_EV','G+/-_x':'G_EV','xG+/-':'xG_EV','TOI_x':'TOI_PP','GAA_x':'GAA_PP','TOI_x':'TOI_SH','GAA_y':'GAA_SH','TOI_y':'TOI'})
#full_player_totals

In [29]:
#Replace NaN with 0
full_player_totals = full_player_totals.replace(np.NaN, 0)
#full_player_totals

In [30]:
#Import Goalie Data
goalie_stats_raw = pd.read_csv('https://raw.githubusercontent.com/danmartin25/Hockey_Model/main/Goalie%20Stats.csv')
#print(goalie_stats_raw)
#goalie_stats_raw.head()

In [31]:
#Reduce Goalie Stats
goalie_stats = goalie_stats_raw.loc[:,['Player','Season','Team','Position','GP','TOI','GA','Sv%','GSAx']]
#goalie_stats.head()

In [32]:
#Add Active / Inactive Column to both data tables
full_player_totals['A/I'] = 'Active'
#full_player_totals

In [33]:
#Make a Data Table for each team
Avalanche_skaters = full_player_totals.loc[(full_player_totals.Team == 'COL')]
Avalanche_skaters

,Player,Season,Team,Position,GP,TOI_SH,xGF%,xGF,xGA,G_EV,xG_EV,TOI,GAA_PP,TOI_SH,GAA_SH,TOI,Points,PAA,Active/Inactive
1221,Alex Newhook,20-21,COL,C,6,74.70,67.70,4.38,2.09,5.32,2.29,97.58,-3.652102,0.00,0.000000,82.07,3,-19.633276,Active
1222,Alex Newhook,20-21,COL,C,6,74.70,67.70,4.38,2.09,5.32,2.29,97.58,-3.652102,0.00,0.000000,963.07,33,10.366724,Active
1223,Alex Newhook,21-22,COL,C,71,842.32,50.05,32.05,31.98,12.63,0.07,97.58,-3.652102,0.00,0.000000,82.07,3,-19.633276,Active
1224,Alex Newhook,21-22,COL,C,71,842.32,50.05,32.05,31.98,12.63,0.07,97.58,-3.652102,0.00,0.000000,963.07,33,10.366724,Active
1662,Andre Burakovsky,19-20,COL,L,58,707.17,49.18,29.04,30.01,14.44,-0.97,136.92,-1.782102,0.00,0.000000,881.17,45,22.366724,Active
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28834,Vladislav Namestnikov,19-20,COL,C,9,118.45,56.20,5.39,4.20,3.42,1.19,44.83,-12.232102,97.72,-3.311789,134.62,6,-16.633276,Active
28835,Vladislav Namestnikov,19-20,COL,C,9,118.45,56.20,5.39,4.20,3.42,1.19,44.83,-12.232102,97.72,-3.311789,816.87,25,2.366724,Active
28836,Vladislav Namestnikov,19-20,COL,C,9,118.45,56.20,5.39,4.20,3.42,1.19,44.83,-12.232102,97.72,-3.311789,856.40,17,-5.633276,Active
28837,Vladislav Namestnikov,19-20,COL,C,9,118.45,56.20,5.39,4.20,3.42,1.19,44.83,-12.232102,97.72,-3.311789,184.92,5,-17.633276,Active


In [43]:
full_player_totals.head(30)

,Player,Season,Team,Position,GP,TOI_SH,xGF%,xGF,xGA,G_EV,xG_EV,TOI,GAA_PP,TOI_SH,GAA_SH,TOI,Points,PAA,Active/Inactive
0,A.J. Greer,21-22,N.J,L,9,70.45,58.74,3.16,2.22,0.06,0.94,0.00,0.000000,0.00,0.000000,70.70,2,-20.633276,Active
1,Aaron Ekblad,19-20,FLA,D,67,1220.55,50.83,49.88,48.26,16.49,1.62,95.90,-2.792102,134.87,-13.441789,1537.92,41,26.890075,Active
2,Aaron Ekblad,19-20,FLA,D,67,1220.55,50.83,49.88,48.26,16.49,1.62,95.90,-2.792102,134.87,-13.441789,878.18,22,7.890075,Active
3,Aaron Ekblad,19-20,FLA,D,67,1220.55,50.83,49.88,48.26,16.49,1.62,95.90,-2.792102,134.87,-13.441789,1519.70,57,42.890075,Active
4,Aaron Ekblad,19-20,FLA,D,67,1220.55,50.83,49.88,48.26,16.49,1.62,95.90,-2.792102,86.47,2.418211,1537.92,41,26.890075,Active
5,Aaron Ekblad,19-20,FLA,D,67,1220.55,50.83,49.88,48.26,16.49,1.62,95.90,-2.792102,86.47,2.418211,878.18,22,7.890075,Active
6,Aaron Ekblad,19-20,FLA,D,67,1220.55,50.83,49.88,48.26,16.49,1.62,95.90,-2.792102,86.47,2.418211,1519.70,57,42.890075,Active
7,Aaron Ekblad,19-20,FLA,D,67,1220.55,50.83,49.88,48.26,16.49,1.62,95.90,-2.792102,166.83,-1.871789,1537.92,41,26.890075,Active
8,Aaron Ekblad,19-20,FLA,D,67,1220.55,50.83,49.88,48.26,16.49,1.62,95.90,-2.792102,166.83,-1.871789,878.18,22,7.890075,Active
9,Aaron Ekblad,19-20,FLA,D,67,1220.55,50.83,49.88,48.26,16.49,1.62,95.90,-2.792102,166.83,-1.871789,1519.70,57,42.890075,Active


In [34]:
#Inspect Data
full_player_totals

,Player,Season,Team,Position,GP,TOI_SH,xGF%,xGF,xGA,G_EV,xG_EV,TOI,GAA_PP,TOI_SH,GAA_SH,TOI,Points,PAA,Active/Inactive
0,A.J. Greer,21-22,N.J,L,9,70.45,58.74,3.16,2.22,0.06,0.94,0.00,0.000000,0.00,0.000000,70.70,2,-20.633276,Active
1,Aaron Ekblad,19-20,FLA,D,67,1220.55,50.83,49.88,48.26,16.49,1.62,95.90,-2.792102,134.87,-13.441789,1537.92,41,26.890075,Active
2,Aaron Ekblad,19-20,FLA,D,67,1220.55,50.83,49.88,48.26,16.49,1.62,95.90,-2.792102,134.87,-13.441789,878.18,22,7.890075,Active
3,Aaron Ekblad,19-20,FLA,D,67,1220.55,50.83,49.88,48.26,16.49,1.62,95.90,-2.792102,134.87,-13.441789,1519.70,57,42.890075,Active
4,Aaron Ekblad,19-20,FLA,D,67,1220.55,50.83,49.88,48.26,16.49,1.62,95.90,-2.792102,86.47,2.418211,1537.92,41,26.890075,Active
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29852,Zemgus Girgensons,19-20,BUF,C,69,783.73,49.54,25.39,25.86,-4.34,-0.47,54.37,-5.272102,81.28,-4.731789,828.20,18,-4.633276,Active
29853,Zemgus Girgensons,21-22,BUF,C/L,56,668.30,45.90,20.39,24.03,-5.14,-3.64,54.37,-5.272102,147.87,-13.281789,951.77,19,-3.633276,Active
29854,Zemgus Girgensons,21-22,BUF,C/L,56,668.30,45.90,20.39,24.03,-5.14,-3.64,54.37,-5.272102,147.87,-13.281789,828.20,18,-4.633276,Active
29855,Zemgus Girgensons,21-22,BUF,C/L,56,668.30,45.90,20.39,24.03,-5.14,-3.64,54.37,-5.272102,81.28,-4.731789,951.77,19,-3.633276,Active


In [35]:
#skater_PP_totals_raw = pd.read_csv('https://raw.githubusercontent.com/danmartin25/Hockey_Model/main/PP%20On-Ice%20Rates.csv')
#print(skater_PP_rates_raw)
#skater_PP_rates_raw.head()

**Stuff to Do Next - (have no more time, so quick thoughts that I have not checked, like the next one would be so easy to check if I had another 5 mins)**

In [36]:
#Are these right? duplicate naming issues for "TOI_SH" and no PP?
full_player_totals.columns

Index(['Player', 'Season', 'Team', 'Position', 'GP', 'TOI_SH', 'xGF%', 'xGF',
       'xGA', 'G_EV', 'xG_EV', 'TOI', 'GAA_PP', 'TOI_SH', 'GAA_SH', 'TOI',
       'Points', 'PAA', 'Active/Inactive'],
      dtype='object')

Need to create a dataframe for each team. Use the loc function on player table first to get all the players for one team. Then do the same thing with goalies. We will probably need to keep them seperate tables since they do not have similar variables

Not necessary at this stage but eventually we are going to want to consolidate these code blocks. Ex, the code to create all of the team rosters should be in one block with no output. For now you can do 64 blocks (32 for skaters, 32 for goalies) and show output so we can see if there are any errors, but once we get past that stage we will comment out the output and put it all in one block. If we ever need to go back in check you just take out the '#' and run the code to see the output.

Need to add a column for 'status'. I am thinking the two states we use are 'active' and 'inactive' for the starting lineups. Will be 18 skaters and 1 goalie when we update daily but doesnt matter now. Should put this in before all of the loc functions so it shows up for the team dataframes. Note: will have to be added to both the full_player_totals dataframe and the goalie_stats dataframe.

Add folder in github for the csv's, update github links in code to pull data